# Generative Adversarial Networks

GANs were first reported in 2014. They have the capability to generate new samples similar to the data they were trained on. For example, creating new faces after being trained on large datasets of faces.

We'll build two networks, a generator and a discriminator. The idea is that we are training a generator to produce samples that the discriminator compares against a sample of real data. Eventually, the generator is able to fool the discriminator into passing a generated sample as real.

The coding is essentially really simple; the real thing is tuning the hyperparameters and the training time involved.

- Discriminator overpowering Generator:

If the discriminator classifies all examples as fake, there is no progress. You may want to have discriminator ouput be unscaled instead of sigmoid.

- Generator overpowering Discriminator: Mode Collapse

The generator discovers some weakness in the discriminator, and the generator ends up producing a similar example regardless of variation in input. You can try adjusting the training rate or changing layers of the discriminator in an attempt to make it better.

GANs are so powerful, they are limited by the hardware that you are using.



# Generate numbers based off the MNIST dataset